#### Fulfillment of Coursera projects

<h2> <u> Table of Contents </u> </h2>

##### [1. Question 1](#Question-1.)  

##### [2. Question 2](#Question-2)

#### Question 1.

In [18]:
from bs4 import BeautifulSoup
import pandas as pd
import requests

In [19]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [20]:
html = requests.get(url) #getting html of the webpage

soup = BeautifulSoup(html.text) ## converting the html to soup element

In [21]:
table = soup.find_all('table', class_='wikitable sortable') ##finding table we need 

In [22]:
header = [x.text.replace('\n', '') for x in table[0].tr.find_all('th')]  # finding header
header

['Postal Code', 'Borough', 'Neighborhood']

In [23]:
#creating pandas data frame 
df = pd.DataFrame(columns=header)
data = {}
for tr in table[0].find_all('tr')[1:]:  #looping through 2 row to end
    td = [x.text.replace('\n', '') for x in tr.find_all('td')] #looping through all the table data in table row
    data['Postal Code'] = td[0]
    data['Borough'] = td[1]
    data['Neighborhood'] = td[2]
    df = df.append(data, ignore_index=True) #appending new row to existing data frame


In [24]:
df.columns = ['PostalCode', 'Borough', 'Neighborhood']  #renameing the columns

In [25]:
df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [26]:
#removing columns with Borough value "Not assigned"
df_clean = df[df.Borough != 'Not assigned'] 

In [27]:
df_clean[list(df_clean.groupby(['PostalCode'])['Neighborhood'].count() != 1)]  #no duplicate value of Postal Code

,PostalCode,Borough,Neighborhood


In [28]:
df_clean.reset_index(drop=True,inplace=True)

In [29]:
df_clean.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [30]:
print(f' There are {df_clean.shape[0]} rows in the data frame')

 There are 103 rows in the data frame


#### Question 2

In [97]:
import geocoder

In [111]:
while(g.latlng is None):
    g = geocoder.google('M5G, Toronto, Ontario')


Status code Unknown from https://maps.googleapis.com/maps/api/geocode/json: ERROR - HTTPSConnectionPool(host='maps.googleapis.com', port=443): Max retries exceeded with url: /maps/api/geocode/json?address=M5G%2C+Toronto%2C+Ontario&bounds=&components=&region=&language= (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x00000094AA1A2F08>, 'Connection to maps.googleapis.com timed out. (connect timeout=5.0)'))


KeyboardInterrupt: 

###### Since geoCoder didn't work in my context.

In [113]:
latlong = pd.read_csv('Geospatial_Coordinates.csv')
latlong.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [116]:
df_ = pd.merge(df_clean, latlong, how='inner', left_on='PostalCode', right_on='Postal Code', right_index=False)

In [119]:
df_clean = df_[['PostalCode', 'Borough', 'Neighborhood', 'Latitude', 'Longitude']]

In [120]:
df_clean.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
